<a href="https://colab.research.google.com/github/preetamjumech/Hadoop_Practice/blob/main/KK_B1_Hadoop_WordCount__06_05_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hadoop
This Notebook has all the codes required to install Hadoop in the Colab VM and execute the a WordCount program using the streaming API <br>
The mapper.py and reducer.py programs are available in the authors G-Drive / Github and are downloaded as required<br>
<hr>


##Acknowledgements
Hadoop Installation from [Anjaly Sam's Github Repository](https://github.com/anjalysam/Hadoop) <br>

To get the concept behind map-reduce see [this notebook](https://github.com/Praxis-QR/BDSN/blob/main/Basic_WordCount_Concept.ipynb) <br>


# 1 Download, Install Hadoop

In [ ]:
# The default JVM available at /usr/lib/jvm/java-11-openjdk-amd64/  works for Hadoop
# But gives errors with Hive https://stackoverflow.com/questions/54037773/hive-exception-class-jdk-internal-loader-classloadersappclassloader-cannot
# Hence this JVM needs to be installed
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# If there is an error in this cell, it is very likely that the version of hadoop has changed
# Download the latest version of Hadoop and change the version numbers accordingly
#wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
#!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
!wget  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
# Unzip it
# the tar command with the -x flag to extract, -z to uncompress, -v for verbose output, and -f to specify that we’re extracting from a file
!tar -xzf hadoop-3.3.2.tar.gz
#copy  hadoop file to user/local
!mv  hadoop-3.3.2/ /usr/local/

--2022-05-06 05:36:04--  https://downloads.apache.org/hadoop/common/hadoop-3.3.2/hadoop-3.3.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638660563 (609M) [application/x-gzip]
Saving to: ‘hadoop-3.3.2.tar.gz’

hadoop-3.3.2.tar.gz 100%[===================>] 609.07M  22.7MB/s    in 27s     

2022-05-06 05:36:31 (22.3 MB/s) - ‘hadoop-3.3.2.tar.gz’ saved [638660563/638660563]



In [ ]:
!ls /usr/local

bin	   cuda-10.1  cuda-11.1  _gcs_config_ops.so  lib	sbin   xgboost
cuda	   cuda-11    etc	 hadoop-3.3.2	     licensing	share
cuda-10.0  cuda-11.0  games	 include	     man	src


# 2 Set Environment Variables


In [ ]:
#To find the default Java path
#!readlink -f /usr/bin/java | sed "s:bin/java::"
#!ls /usr/lib/jvm/

In [ ]:
#To set java path, go to /usr/local/hadoop-3.3.0/etc/hadoop/hadoop-env.sh then
#. . . export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/ . . .
#we have used a simpler alternative route using os.environ - it works

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"   # default is changed
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
# make sure that the version number is as downloaded 
#os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.0/"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.3.2/"

In [ ]:
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [ ]:
# Add Hadoop BIN to PATH
# Get the current_path from output of previous command
#current_path = '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'
current_path = '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'
new_path = current_path+':/usr/local/hadoop-3.3.2/bin/'
os.environ["PATH"] = new_path
!echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/usr/local/hadoop-3.3.2/bin/


# 3 Test Hadoop Installation

In [ ]:
#Running Hadoop - Test RUN, not doing anything at all
#!/usr/local/hadoop-3.3.0/bin/hadoop
# UNCOMMENT the following line if you want to make sure that Hadoop is alive!
!hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [ ]:
# Testing Hadoop with PI generating sample program, should calculate value of pi = 3.14157500000000000000
# pi example
#Uncomment the following line if  you want to test Hadoop with pi example
# Final output should be : Estimated value of Pi is 3.14157500000000000000
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar pi 16 100000
!hadoop jar /usr/local/hadoop-3.3.2/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.2.jar pi 16 100000

Number of Maps  = 16
Samples per Map = 100000
Wrote input for Map #0
Wrote input for Map #1
Wrote input for Map #2
Wrote input for Map #3
Wrote input for Map #4
Wrote input for Map #5
Wrote input for Map #6
Wrote input for Map #7
Wrote input for Map #8
Wrote input for Map #9
Wrote input for Map #10
Wrote input for Map #11
Wrote input for Map #12
Wrote input for Map #13
Wrote input for Map #14
Wrote input for Map #15
Starting Job
2022-05-06 05:45:51,546 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-05-06 05:45:51,666 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-05-06 05:45:51,666 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-05-06 05:45:51,839 INFO input.FileInputFormat: Total input files to process : 16
2022-05-06 05:45:51,856 INFO mapreduce.JobSubmitter: number of splits:16
2022-05-06 05:45:52,124 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local961884597_0001
2022-05-06 05:4

# 4 Run WordCount with Hadoop
Instead of using Java for Map and Reduce methods, we use the streaming API of Hadoop and two simple python programs as mapper.py and reducer.py

In [ ]:
# get mapper.py reducer.py from G_drive
#!gdown https://drive.google.com/uc?id=1VTzQ18cWAj6L29ncW6sABy-ITmDCcv5r
#!gdown https://drive.google.com/uc?id=1Or8Cbf9AsFMHStjMzDw3pXCd6TZ0dqxJ

#get mapper.py reducer.py from this git repository
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/main/mapper.py
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/main/reducer.py

In [ ]:
# to see the codes, uncomment the following lines
#!cat mapper.py
#print("\n----------------------    see above for mapper, see below for reducer")
#!cat reducer.py

In [ ]:
# python codes are made executable
!chmod u+rwx /content/mapper.py
!chmod u+rwx /content/reducer.py

In [ ]:
# get a simple txt file as data for word count
# or you can upload your own
#!gdown https://drive.google.com/uc?id=1R5W0UVH2S3JjPxerqyX4ue5y6tMt0Wkk
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/main/Chronotantra.txt

In [ ]:
# locate the streaming jar file
!find / -name 'hadoop-streaming*.jar'

find: ‘/proc/28/task/28/net’: Invalid argument
find: ‘/proc/28/net’: Invalid argument
/usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar
/usr/local/hadoop-3.3.2/share/hadoop/tools/sources/hadoop-streaming-3.3.2-test-sources.jar
/usr/local/hadoop-3.3.2/share/hadoop/tools/sources/hadoop-streaming-3.3.2-sources.jar


In [ ]:
# remove output directories
!rm -r wc_out
!rm -r wc2_out

rm: cannot remove 'wc_out': No such file or directory
rm: cannot remove 'wc2_out': No such file or directory


In [ ]:
# execute the streaming jar with proper parameters
# four parameters are input file, output directory, the mapper progra, the reducer program
#
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/hobbit.txt -output /content/wc_out -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/Chronotantra.txt -output /content/wc_out -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'
#!hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/Chronotantra.txt -output /content/wc_out  -mapper 'python mapper.py'  -reducer 'python reducer.py'
!hadoop jar /usr/local/hadoop-3.3.2/share/hadoop/tools/lib/hadoop-streaming-3.3.2.jar -input /content/Chronotantra.txt -output /content/wc_out  -mapper 'python mapper.py'  -reducer 'python reducer.py'

2022-05-06 05:52:01,533 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-05-06 05:52:01,651 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-05-06 05:52:01,651 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-05-06 05:52:01,671 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-05-06 05:52:01,883 INFO mapred.FileInputFormat: Total input files to process : 1
2022-05-06 05:52:01,907 INFO mapreduce.JobSubmitter: number of splits:1
2022-05-06 05:52:02,145 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local575339868_0001
2022-05-06 05:52:02,145 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-05-06 05:52:02,368 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2022-05-06 05:52:02,370 INFO mapreduce.Job: Running job: job_local575339868_0001
2022-05-06 05:52:02,377 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2022-05-0

In [ ]:
# check output directory
!ls wc_out

part-00000  _SUCCESS


In [ ]:
# see actual output
#!tail wc_out/part-00000
!head wc_out/part-00000

1	8
10	2
100	2
1000	1
105	1
108	2
109	1
11	1
110	2
113	1


### Sorting the output

In [ ]:
#https://www.geeksforgeeks.org/sort-command-linuxunix-examples/
!sort -nr -k 2 -t$'\t' wc_out/part-00000 > sorted.txt

In [ ]:
!head -30 sorted.txt


would	346
could	247
one	198
time	156
like	145
know	144
us	134
mars	119
back	106
even	105
world	97
something	95
see	95
well	93
hermit	93
two	87
people	86
course	84
around	84
way	82
first	80
really	79
new	76
little	74
long	73
still	71
information	70
ai	67
good	63
earth	60


In [ ]:
!tail -30 sorted.txt

2150	1
214	1
206	1
205	1
2019	1
2018	1
2007	1
20062007	1
2000	1
1999	1
1970s	1
1956	1
187	1
186	1
17866	1
156	1
155	1
15	1
150	1
1493	1
133	1
132	1
12th	1
12700	1
115	1
113	1
11	1
109	1
105	1
1000	1
